In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

import rf

import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

In [ ]:
# References:
# Yu, Y., J. Song, K. H. Liu, and S. S. Gao (2015), Determining crustal structure beneath seismic stations overlying a low-velocity
# sedimentary layer using receiver functions, J. Geophys. Res. Solid Earth, 120, 3208–3218, doi:10.1002/2014JB011610.
# http://dx.doi.org/10.1002/2014JB011610

## Data loading

In [ ]:
rf_type = rf_type = 'ZRT_it'

In [ ]:
network = 'AQ'
target_station = 'D7'

In [ ]:
src_file = r"..\DATA\AQT_rfs_20151128T042000-20191108T000317_{}_rev1_qual.h5".format(rf_type)

In [ ]:
data_all = rf_util.read_h5_rf(src_file, network, target_station)

In [ ]:
data_all

In [ ]:
type(data_all)

## Select R-channel data

In [ ]:
db = rf_util.rf_to_dict(data_all)

In [ ]:
station_db = db[target_station]

In [ ]:
channel = rf_util.choose_rf_source_channel(rf_type, station_db)
print("Selected channel: {}".format(channel))
channel_data = station_db[channel]
len(channel_data)

In [ ]:
type(channel_data)

In [ ]:
type(channel_data[0])

In [ ]:
channel_data[0].stats.sampling_rate

## Plot initial data

In [ ]:
_ = rf_plot_utils.plot_rf_stack(rf.RFStream(channel_data))

## Look at initial H-k stack

In [ ]:
k_grid, h_grid, hk = rf_stacking.compute_hk_stack(channel_data, include_t3=False, root_order=2, k_range=np.linspace(1.5, 2.0, 301))
w = (0.5, 0.5)
w_str = "w={:1.2f},{:1.2f}".format(*w)
stack = rf_stacking.compute_weighted_stack(hk, weighting=w)
hk_fig = rf_plot_utils.plot_hk_stack(k_grid, h_grid, stack, num=len(channel_data),
                                     title="Initial H-k stack, reverb ({})".format(w_str))
plt.show()

## Trial reverb removal

In [ ]:
x = np.array([d for d in channel_data])

In [ ]:
type(x)

In [ ]:
x.shape

In [ ]:
ac = np.apply_along_axis(lambda _x: np.correlate(_x, _x, 'full')/np.dot(_x, _x), axis=1, arr=x)
ac.shape

In [ ]:
ac_causal = ac[:, ac.shape[1]//2:]
ac_causal.shape

In [ ]:
plt.plot(ac_causal[0, :500])

In [ ]:
plt.stem(ac_causal[0, :50])

In [ ]:
ac_extrema = np.apply_along_axis(lambda _x: signal.argrelextrema(_x, np.less)[0][0], axis=1, arr=ac_causal)

In [ ]:
ac_extrema

In [ ]:
def reverb_removal_filter(offset, amplitude):
    reverb_removal_kernel = np.zeros(offset + 1)
    reverb_removal_kernel[0] = 1
    reverb_removal_kernel[-1] = amplitude
    return reverb_removal_kernel

In [ ]:
x_norev = [np.convolve(_x, reverb_removal_filter(ac_extrema[i], -ac_causal[i, ac_extrema[i]]), 'full') for i, _x in enumerate(x)]

In [ ]:
x_norev = [_x[:(-ac_extrema[i] + 1)] for i, _x in enumerate(x_norev)]

## Construct and plot RF stream from de-reverbed traces

In [ ]:
traces = []
for i, tr in enumerate(channel_data):
    new_tr = tr.copy()
    new_tr.data = x_norev[i]
    traces.append(new_tr)
# end for
channel_data_noreverb = rf.RFStream(traces)

In [ ]:
_ = rf_plot_utils.plot_rf_stack(rf.RFStream(channel_data_noreverb))

## Look at H-k stacking of de-reverbed traces

In [ ]:
k_grid, h_grid, hk = rf_stacking.compute_hk_stack(channel_data_noreverb, include_t3=False, root_order=2, k_range=np.linspace(1.5, 2.0, 301))
w = (0.5, 0.5)
w_str = "w={:1.2f},{:1.2f}".format(*w)
stack = rf_stacking.compute_weighted_stack(hk, weighting=w)
hk_fig = rf_plot_utils.plot_hk_stack(k_grid, h_grid, stack, num=len(channel_data_noreverb),
                                     title="Refined H-k stack, reduced reverb ({})".format(w_str))
plt.show()

In [ ]:
# NEXT: Continue process in reference